In [1]:
import pandas as pd

In [2]:
gt_test = pd.DataFrame(pd.read_csv('annotation_test.csv'))
gt_test.head()

,no,x,y,w,h,contrast
0,Sample_1\frame_0009.png,1194,689,5,5,10
1,Sample_1\frame_0055.png,1138,642,5,5,10
2,Sample_1\frame_0060.png,1133,638,5,5,10
3,Sample_1\frame_0077.png,1116,611,5,5,10
4,Sample_1\frame_0082.png,1113,612,5,5,10


In [33]:
gt_dict = {}
for data in gt_test.itertuples():
    gt_dict[data[1]] = (data[2],data[3],data[4],data[5])
len(gt_dict)

4206

In [14]:
def replace_sep(s):
    return s.replace('/','\\')

In [16]:
test_res = pd.DataFrame(pd.read_csv('unet_results.csv')).iloc[:,1:]
test_res['Sample'] = test_res['Sample'].apply(replace_sep)
test_res.head()

,Sample,x,y,w,h
0,Sample_39\frame_0617.png,629,67,5,5
1,Sample_2\frame_0595.png,627,131,5,5
2,Sample_44\frame_0186.png,1011,523,5,5
3,Sample_51\frame_0328.png,885,345,5,5
4,Sample_47\frame_0336.png,862,347,5,5


In [47]:
test_dict = {}
for data in test_res.itertuples():
    if data[1] in test_dict:
        test_dict[data[1]].append((data[2],data[3],data[4],data[5]))
    else:
        test_dict[data[1]] = [(data[2],data[3],data[4],data[5])]
len(test_dict)

4141

In [40]:
def IoU(b1, b2):
    x1,y1,w1,h1 = b1
    x2,y2,w2,h2 = b2
    xA = max(x1, x2)
    yA = max(y1, y2)
    xB = min(x1+w1, x2+w2)
    yB = min(y1+h1, y2+h2)

    inter = max(0, xB - xA) * max(0, yB - yA)
    
    return inter/(w1*h1+w2*h2-inter)

In [41]:
result_df = {'Sample':[],'no_detections':[],'iou=1':[], 'iou>0.5':[], 'iou>0':[],'IoU':[],'false_alarms':[]}
for sample in gt_dict:
    result_df['Sample'].append(sample)
    if sample not in test_dict:
        for col in result_df:
            if col == 'Sample':
                continue
            else:
                result_df[col].append(0)
    else:
        result_df['no_detections'].append(len(test_dict[sample]))
        IoU_val = 0
        b_gt = gt_dict[sample]
        for bbox in test_dict[sample]:
            IoU_val += IoU(b_gt,bbox)
        result_df['IoU'].append(IoU_val)
        result_df['iou=1'].append(int(IoU_val>0.99))
        result_df['iou>0.5'].append(int(IoU_val>0.5))
        result_df['iou>0'].append(int(IoU_val>0.01))
        result_df['false_alarms'].append(len(test_dict[sample]) - int(IoU_val>0.01))
            
result_df = pd.DataFrame(result_df)
result_df.head()

,Sample,no_detections,iou=1,iou>0.5,iou>0,IoU,false_alarms
0,Sample_1\frame_0009.png,1,1,1,1,1.0,0
1,Sample_1\frame_0055.png,1,1,1,1,1.0,0
2,Sample_1\frame_0060.png,1,1,1,1,1.0,0
3,Sample_1\frame_0077.png,1,1,1,1,1.0,0
4,Sample_1\frame_0082.png,1,1,1,1,1.0,0


In [66]:
result_df.loc[result_df.no_detections == 0].shape[0]/len(gt_dict)*100 # no detection rate

1.5454113171659534

In [67]:
result_df.loc[result_df.no_detections > 1].shape[0]/len(gt_dict)*100 # more than 1 detection rate

0.11887779362815026

In [68]:
result_df.sum(axis = 0)['iou=1']/len(gt_dict)*100 #iou = 1 rate

92.93865905848787

In [69]:
result_df.sum(axis = 0)['iou>0.5']/len(gt_dict)*100 #iou > 0.5 rate

98.14550641940085

In [70]:
result_df.sum(axis = 0)['iou>0']/len(gt_dict)*100 #successfully detected rate

98.43081312410843

In [71]:
result_df.sum(axis = 0)['IoU']/len(gt_dict)*100 #avg IoU

97.11023866671411

In [72]:
result_df.sum(axis = 0)['false_alarms']/len(gt_dict)*100 #false alarm rate

0.14265335235378032